In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
import csv
from collections import OrderedDict

In [6]:
#定义读取格式
class testDataset(Dataset):
    def __init__(self,test_INFO,test_DATA,cropsize=32, transform=None):
        index = []
        index += list(test_INFO.index)
        self.index = tuple(sorted(index))
        self.transform = transform
        self.cropsize = cropsize

    def __getitem__(self, item):
        name = test_INFO.loc[self.index[item], 'name']
        npz_temp = np.load(os.path.join(test_DATA, '%s.npz' % name))
        seg_temp = npz_temp['seg']+0
        voxel_temp = npz_temp['voxel']
        seg_temp = seg_temp[50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2]
        voxel_temp = voxel_temp[50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2,
                     50 - self.cropsize // 2:50 + self.cropsize // 2]
        data=np.multiply(seg_temp,voxel_temp)
        data = data[np.newaxis, ...]
        #seg = seg_temp[np.newaxis, ...]
        return name, data
    
    def __len__(self):
        return len(self.index)

class ToTensor(object):
    def __call__(self, sample):

        return torch.from_numpy(sample)

In [7]:
#定义数据路径
os.chdir("./")#当前路径
DATASET="./dataset/"
test_INFO = pd.read_csv(os.path.join(DATASET, "sampleSubmission.csv"))
#test_INFO = pd.read_csv("sampleSubmission(1).csv")
test_DATA= os.path.join(DATASET, 'test')
#test_DATA= 'test(1)'
#读取数据并转成tensor
test_dataset = testDataset(test_INFO,test_DATA,cropsize=32, transform=ToTensor())
#封装成dataloader的格式
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0,drop_last=False)

In [8]:
class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):    # bn_size need to be 4
        super(_DenseLayer, self).__init__()
        self.add_module('norm1', nn.BatchNorm3d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv3d(num_input_features, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm2', nn.BatchNorm3d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv3d(bn_size * growth_rate, growth_rate,kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm3d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv3d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool3d(kernel_size=2, stride=2))

class DenseNet(nn.Module):
    def __init__(self, growth_rate=16, block_config=(4, 4, 4),
                 num_init_features=32, bn_size=4, drop_rate=0, num_classes=2):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv3d(1, num_init_features, kernel_size=3, stride=1, padding=0, bias=False)),
            ('norm0', nn.BatchNorm3d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
        ]))

        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features, num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm3d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.avg_pool3d(out, kernel_size=7, stride=1).view(features.size(0), -1)
        out = self.classifier(out)
        out = F.softmax(out, dim=1)
        return out
    
    
model=DenseNet();
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [10]:
csvFile = open("test_submit.csv", "w", newline='')
writer = csv.writer(csvFile)
writer.writerow(["Id", "Predicted"])
model = torch.load('model.pth', map_location='cpu')
model.eval()
device = torch.device("cpu")
model.to(device)
with torch.no_grad():
    for ID,data in test_loader:
        data = data.type(torch.FloatTensor)
        data = data.to(device)
        outputs = model(data)
        outputs=torch.softmax(outputs,1)
        writer.writerow([str(ID[0]), (outputs.data[0][1]).item()])
            #print(str(ID[0]), (outputs.data[0][0]).item(), (outputs.data[0][1]).item())
csvFile.close()